In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110580  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,9.971206e+06
2018-02-28,8.618192e+06
2018-03-31,9.169547e+06
2018-04-30,1.020908e+07
2018-05-31,1.054542e+07
2018-06-30,9.964301e+06
2018-07-31,1.088450e+07
2018-08-31,1.159131e+07
2018-09-30,1.011676e+07


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    9.971206e+06
 2018-02-28    8.618192e+06
 2018-03-31    9.169547e+06
 2018-04-30    1.020908e+07
 2018-05-31    1.054542e+07
 2018-06-30    9.964301e+06
 2018-07-31    1.088450e+07
 2018-08-31    1.159131e+07
 2018-09-30    1.011676e+07
 2018-10-31    1.052433e+07
 2018-11-30    9.877914e+06
 2018-12-31    9.555051e+06
 2019-01-31    1.057873e+07
 2019-02-28    1.017920e+07
 2019-03-31    1.029889e+07
 2019-04-30    1.095005e+07
 2019-05-31    1.250201e+07
 2019-06-30    1.185333e+07
 2019-07-31    1.209752e+07
 2019-08-31    1.235822e+07
 2019-09-30    1.043018e+07
 2019-10-31    1.217698e+07
 2019-11-30    1.134339e+07
 2019-12-31    1.153333e+07
 2020-01-31    1.219792e+07
 2020-02-29    7.220891e+06
 2020-03-31    6.631922e+06
 2020-04-30    6.458796e+06
 2020-05-31    8.320115e+06
 2020-06-30    9.299853e+06
 2020-07-31    1.054832e+07
 2020-08-31    1.148873e+07
 2020-09-30    8.435364e+06
 2020-10-31    1.011857e+07
 2020-11-30    9.500663e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.728568
p-value : 0.069224
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -6.960232
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1326.781, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1311.566, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1312.230, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1306.820, Time=0.09 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1310.258, Time=0.09 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1311.108, Time=0.20 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1303.712, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1305.657, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1307.861, Time=0.12 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1307.527, Time=0.13 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1306.954, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1305.541, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1307.591, Time=0.14 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1305.835, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1308.601, Time=0.23 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -648.841
Method:                       css-mle   S.D. of innovations        1230311.314
Date:                Sat, 11 Sep 2021   AIC                           1303.682
Time:                        02:01:42   BIC                           1308.895
Sample:                             0   HQIC                          1305.593
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.857e+06   5.38e+05     18.329      0.000     8.8e+06    1.09e+07
ar.L1.y        0.6624      0.111      5.984      0.000       0.445       0.879
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([9434850.81512987, 9577380.02354533]), array([1230311.31402974, 1475736.1282072 ]), array([[ 7023484.94985942, 11846216.68040032],
       [ 6684990.36157464, 12469769.68551603]]))
